In [11]:
import logging
import geopandas as gpd
from deafrica_waterbodies.cli.logs import logging_setup

In [12]:
import os

# These are the default AWS configurations for the Analysis Sandbox.
# that are set in the environmnet variables.
aws_default_config = {
    # "AWS_NO_SIGN_REQUEST": "YES",
    "AWS_SECRET_ACCESS_KEY": "fake",
    "AWS_ACCESS_KEY_ID": "fake",
}

# To access public bucket, need to remove the AWS credentials in
# the environment variables or the following error will occur.
# PermissionError: The AWS Access Key Id you provided does not exist in our records.

for key in aws_default_config.keys():
    if key in os.environ:
        del os.environ[key]

In [13]:
verbose = 1
output_directory = "s3://deafrica-waterbodies-dev/test_out_dir"

In [14]:
# Set up logger.
logging_setup(verbose=verbose)
_log = logging.getLogger(__name__)

In [15]:
# Support pathlib paths.
output_directory = str(output_directory)

In [16]:
_log.info("Loading filtered waterbody polygons...")
polygons_fp = os.path.join(output_directory, "large_polygons_handled.parquet")

polygons = gpd.read_parquet(polygons_fp)
_log.info(f"Waterbody polygons count {len(primary_threshold_polygons)}.")

[2023-10-04 20:33:07,115] {196592127.py:1} INFO - Loading filtered waterbody polygons...


FileNotFoundError: deafrica-waterbodies-dev/test_out_dir/large_polygons_handled.parquet